In [1]:
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [2]:
model_path = 'D:\\ProgPrj\\mezhnar\\data\\learning\\runs\\detect\\train7222\\weights\\best.pt'
model_pt = YOLO(model_path)

In [163]:
img_path = 'D:\\ProgPrj\\mezhnar\\data\\splited_data\\valid\\images\\21.JPG'
img = cv2.imread(img_path)

In [164]:
results = model_pt(img_path)


image 1/1 D:\ProgPrj\mezhnar\data\splited_data\valid\images\21.JPG: 640x480 1 0, 155.9ms
Speed: 3.0ms preprocess, 155.9ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 480)


In [166]:
import os
import math  # Для проверки NaN значений
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Создаем папку 'rotate', если она еще не существует
output_folder = 'D:\\ProgPrj\\mezhnar\\research\\rotate'
os.makedirs(output_folder, exist_ok=True)

best_confidence = -1  # Начальное значение для хранения лучшей вероятности
best_cropped_image = None  # Переменная для хранения лучшего обрезанного изображения

for result in results:
    original_image = result.orig_img  # Получаем оригинальное изображение без bbox

    # Проверяем, есть ли у result атрибут boxes и обрабатываем bbox, если он есть
    if hasattr(result, 'boxes'):
        total_boxes = len(result.boxes)
        print(f"Total bounding boxes: {total_boxes}")

        for i, box in enumerate(result.boxes):
            # Проверяем, что box имеет атрибут xyxy для извлечения координат
            if hasattr(box, 'xyxy') and box.xyxy.shape[-1] == 4:
                x_min, y_min, x_max, y_max = box.xyxy[0].tolist()

                # Попытка получить значение вероятности
                confidence = None
                if hasattr(box, 'confidence'):
                    confidence = box.confidence.item() * 100
                elif hasattr(box, 'score'):
                    confidence = box.score.item() * 100
                elif hasattr(box, 'conf'):
                    confidence = box.conf.item() * 100

                # Проверяем, что значение confidence не NaN и больше текущего лучшего значения
                if confidence is not None and not math.isnan(confidence):
                    print(f"Bounding Box {i + 1} Probability: {confidence:.2f}%")

                    if confidence > best_confidence:
                        best_confidence = confidence

                        # Расширяем рамки с учетом padding и обрезаем изображение
                        padding = 0.1  # Процент увеличения рамки
                        height, width, _ = original_image.shape
                        x_min = max(0, int(x_min - padding * (x_max - x_min)))
                        y_min = max(0, int(y_min - padding * (y_max - y_min)))
                        x_max = min(width, int(x_max + padding * (x_max - x_min)))
                        y_max = min(height, int(y_max + padding * (y_max - y_min)))

                        # Обрезаем изображение по расширенным рамкам и сохраняем как лучшее
                        best_cropped_image = original_image[y_min:y_max, x_min:x_max]
                else:
                    print(f"Ошибка: Probability для Bounding Box {i + 1} является NaN или отсутствует.")
    else:
        print("Ошибка: result не содержит атрибут boxes.")

# Сохраняем оригинал и три поворота, если изображение было найдено
if best_cropped_image is not None:
    # Конвертируем обрезанное изображение в формат PIL для сохранения
    best_cropped_image_pil = Image.fromarray(best_cropped_image)
    best_cropped_image_pil.save(os.path.join(output_folder, "original.png"))

    # Вращаем и сохраняем изображение три раза
    for rotation in range(1, 4):
        rotated_image = np.rot90(best_cropped_image, rotation)
        rotated_image_pil = Image.fromarray(rotated_image)
        rotated_image_pil.save(os.path.join(output_folder, f"rotated_{rotation * 90}.png"))
else:
    print("Нет подходящего изображения с уверенностью выше порога.")


Total bounding boxes: 1
Bounding Box 1 Probability: 85.48%


#### Easy OCR

In [167]:
import easyocr
import cv2

In [168]:
reader = easyocr.Reader(['en', 'ru'])

In [169]:
pp = 'D:\\ProgPrj\\mezhnar\\research\\rotate\\original.png'
pp90 = 'D:\\ProgPrj\\mezhnar\\research\\rotate\\rotated_90.png'
pp180 = 'D:\\ProgPrj\\mezhnar\\research\\rotate\\rotated_180.png'
pp270 = 'D:\\ProgPrj\\mezhnar\\research\\rotate\\rotated_270.png'

In [170]:
image = cv2.imread(pp)
results_ocr = reader.readtext(image)

In [171]:
for(bbox, text, prob) in results_ocr:
    print(text, prob)

175 0.490319162607193


In [172]:
image = cv2.imread(pp90)
results_ocr = reader.readtext(image)

In [173]:
for(bbox, text, prob) in results_ocr:
    print(text, prob)

In [174]:
image = cv2.imread(pp180)
results_ocr = reader.readtext(image)

In [175]:
for(bbox, text, prob) in results_ocr:
    print(text, prob)

#ElY 0.0038454311434179544


In [176]:
image = cv2.imread(pp270)
results_ocr = reader.readtext(image)

In [177]:
for(bbox, text, prob) in results_ocr:
    print(text, prob)